In [1]:
import logging
import sys

from naslib.defaults.trainer import Trainer

from naslib.search_spaces import (
    DartsSearchSpace,
    SimpleCellSearchSpace,
    NasBench101SearchSpace,
    HierarchicalSearchSpace,
)
from naslib.search_spaces.nasbench101 import graph

from naslib.utils import get_dataset_api, setup_logger, utils

import numpy as np

device: cpu
device: cpu
device: cpu
device: cpu
device: cpu
device: cpu


### Defining the Dataset

In [2]:
config = utils.get_config_from_args(config_type="nas")

In [3]:
dataset_api = get_dataset_api(config.search_space, config.dataset)

Loading dataset from file... This may take a few minutes...
Loaded dataset in 5 seconds


### Defining the Global Variables

In [4]:
INPUT = "input"
OUTPUT = "output"
CONV3X3 = "conv3x3-bn-relu"
CONV1X1 = "conv1x1-bn-relu"
MAXPOOL3X3 = "maxpool3x3"
OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]

NUM_VERTICES = 7
OP_SPOTS = NUM_VERTICES - 2
MAX_EDGES = 9

In [5]:
def sample_random_architecture(dataset_api, arch_limit=10):
        """
        This will sample a random architecture and update the edges in the
        naslib object accordingly.
        From the NASBench repository:
        one-hot adjacency matrix
        draw [0,1] for each slot in the adjacency matrix
        """
        architectures = []
        while len(architectures) < arch_limit:
            matrix = np.random.choice([0, 1], size=(NUM_VERTICES, NUM_VERTICES))
            matrix = np.triu(matrix, 1)
            ops = np.random.choice(OPS, size=NUM_VERTICES).tolist()
            ops[0] = INPUT
            ops[-1] = OUTPUT
            spec = dataset_api["api"].ModelSpec(matrix=matrix, ops=ops)
            if dataset_api["nb101_data"].is_valid(spec):
                architectures.append({"matrix": matrix, "ops": ops})
                #break
        
        return architectures
            
        #self.set_spec({"matrix": matrix, "ops": ops})

In [6]:
sampled_architectures = sample_random_architecture(dataset_api)

In [7]:
sampled_architectures

[{'matrix': array([[0, 0, 1, 1, 0, 0, 1],
         [0, 0, 0, 1, 0, 0, 1],
         [0, 0, 0, 0, 1, 0, 1],
         [0, 0, 0, 0, 1, 1, 1],
         [0, 0, 0, 0, 0, 1, 1],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0]]),
  'ops': ['input',
   'maxpool3x3',
   'maxpool3x3',
   'maxpool3x3',
   'conv1x1-bn-relu',
   'conv3x3-bn-relu',
   'output']},
 {'matrix': array([[0, 0, 1, 0, 1, 0, 0],
         [0, 0, 1, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 1, 1],
         [0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0]]),
  'ops': ['input',
   'conv1x1-bn-relu',
   'conv1x1-bn-relu',
   'conv1x1-bn-relu',
   'conv1x1-bn-relu',
   'conv3x3-bn-relu',
   'output']},
 {'matrix': array([[0, 0, 0, 0, 1, 0, 1],
         [0, 0, 1, 0, 1, 0, 0],
         [0, 0, 0, 0, 1, 0, 1],
         [0, 0, 0, 0, 1, 1, 0],
         [0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 0, 0]]),
  'ops': ['input',


In [8]:
sampled_architectures[0]

{'matrix': array([[0, 0, 1, 1, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]),
 'ops': ['input',
  'maxpool3x3',
  'maxpool3x3',
  'maxpool3x3',
  'conv1x1-bn-relu',
  'conv3x3-bn-relu',
  'output']}

In [9]:
from naslib.optimizers import (
    DARTSOptimizer,
    GDASOptimizer,
    DrNASOptimizer,
    RandomSearch,
    RegularizedEvolution,
    LocalSearch,
    BasePredictor,
)

In [10]:
optimizer = RandomSearch(config)

In [11]:
search_space = NasBench101SearchSpace()

Update function could not be verified. Be cautious with the setting of `private_edge_data` in `update_edges()`
Update function could not be verified. Be cautious with the setting of `private_edge_data` in `update_edges()`
Update function could not be verified. Be cautious with the setting of `private_edge_data` in `update_edges()`
Update function could not be verified. Be cautious with the setting of `private_edge_data` in `update_edges()`
Update function could not be verified. Be cautious with the setting of `private_edge_data` in `update_edges()`


In [12]:
optimizer.adapt_search_space(search_space, dataset_api=dataset_api)

In [14]:
model = optimizer.new_epoch(108)

In [20]:
model

Module(
  (arch): Graph makrograph-0.2958464, scope macro, 6 nodes
)

In [23]:
optimizer.sampled_archs[0]

Module(
  (arch): Graph makrograph-0.2958464, scope macro, 6 nodes
)

In [24]:
from networkx.algorithms.dag import lexicographical_topological_sort

In [25]:
import torch

In [26]:
for node_idx in lexicographical_topological_sort(model):
            if "subgraph" in model.nodes[node_idx]:
                model.nodes[node_idx]["subgraph"].parse()
                model.add_module(
                    "{}-subgraph_at({})".format(model.name, node_idx),
                    model.nodes[node_idx]["subgraph"],
                )
            else:
                if isinstance(model.nodes[node_idx]["comb_op"], torch.nn.Module):
                    model.add_module(
                        "{}-comb_op_at({})".format(model.name, node_idx),
                        model.nodes[node_idx]["comb_op"],
                    )
            for neigbor_idx in model.neighbors(node_idx):
                edge_data = model.get_edge_data(node_idx, neigbor_idx)
                edge_data.op.parse()
                if edge_data.op.get_embedded_ops():
                    for primitive in edge_data.op.get_embedded_ops():
                        primitive.parse()
                model.add_module(
                    "{}-edge({},{})".format(model.name, node_idx, neigbor_idx),
                    edge_data.op,
                )
        #model.is_parsed = True

AttributeError: 'Module' object has no attribute 'is_directed'